<a href="https://colab.research.google.com/github/alessioborgi/AMR_CleaningRobot/blob/master/X_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# X-GNN

### 0: IMPORTING LIBRARIES

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00


In [8]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

# Define a single Graph Convolution layer
class GraphConvolution(nn.Module):
    """
    Implements a simple GCN layer as described in the paper:
    'Semi-Supervised Classification with Graph Convolutional Networks' (https://arxiv.org/abs/1609.02907)
    """

    def __init__(self, in_features, out_features):
        """
        Initialize the GCN layer with input and output feature dimensions.

        Args:
        - in_features: Number of input features per node.
        - out_features: Number of output features per node.
        """
        super(GraphConvolution, self).__init__()
        self.in_features = in_features  # Number of input features
        self.out_features = out_features  # Number of output features

        # Learnable weight matrix of shape [in_features, out_features]
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        # Learnable bias vector of shape [out_features]
        self.bias = Parameter(torch.FloatTensor(out_features))

        # Initialize weights and bias
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize weights and bias using uniform distribution based on feature dimensions.
        """
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)  # Uniform initialization for weights
        self.bias.data.uniform_(-stdv, stdv)    # Uniform initialization for bias

    def __repr__(self):
        """
        Return a string representation of the layer showing the input and output feature dimensions.
        """
        return self.__class__.__name__ + f' ({self.in_features} -> {self.out_features})'

    def forward(self, input, adj):
        """
        Forward pass of the GCN layer: computes A~XW + b.

        Args:
        - input: Node feature matrix X of shape [num_nodes, in_features].
        - adj: Normalized adjacency matrix A~ of shape [num_nodes, num_nodes].

        Returns:
        - Output feature matrix of shape [num_nodes, out_features].
        """
        # Compute XW
        support = torch.mm(input, self.weight)
        # Compute A~XW using sparse matrix multiplication
        output = torch.spmm(adj, support)
        # Add bias and return
        return output + self.bias

# Define the full Graph Convolutional Network (GCN)
class GCN(nn.Module):
    """
    Implements a 3-layer GCN model with a final classifier.
    """

    def __init__(self, nfeat, nclass, dropout):
        """
        Initialize the GCN model.

        Args:
        - nfeat: Number of input features per node.
        - nclass: Number of output classes (for classification).
        - dropout: Dropout rate for regularization.
        """
        super(GCN, self).__init__()

        self.dropout = dropout  # Dropout rate

        # Define three GCN layers
        self.gc1 = GraphConvolution(nfeat, 32)  # Layer 1: Input to 32 features
        self.gc2 = GraphConvolution(32, 48)    # Layer 2: 32 to 48 features
        self.gc3 = GraphConvolution(48, 64)    # Layer 3: 48 to 64 features

        # Fully connected layers for classification
        self.fc1 = nn.Linear(64, 32)  # Hidden layer: 64 to 32 features
        self.fc2 = nn.Linear(32, nclass)  # Output layer: 32 to nclass

    def forward(self, x, adj):
        """
        Forward pass of the GCN model.

        Args:
        - x: Node feature matrix X of shape [num_nodes, nfeat].
        - adj: Normalized adjacency matrix A~ of shape [num_nodes, num_nodes].

        Returns:
        - Output class probabilities of shape [nclass].
        """
        # Apply first GCN layer with ReLU activation
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)  # Apply dropout

        # Apply second GCN layer with ReLU activation
        x = F.relu(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)  # Apply dropout

        # Apply third GCN layer with ReLU activation
        x = F.relu(self.gc3(x, adj))

        # Aggregate node features using mean pooling
        y = torch.mean(x, dim=0)  # Aggregate across nodes

        # Pass through fully connected layers with ReLU and dropout
        y = F.relu(self.fc1(y))
        y = F.dropout(y, self.dropout, training=self.training)
        y = F.softmax(self.fc2(y), dim=0)  # Output class probabilities

        return y

# Example usage
if __name__ == '__main__':
    # Create a random node feature matrix (29 nodes, 7 features per node)
    input = torch.rand(29, 7)
    # Create a random adjacency matrix (29 nodes)
    adj = torch.rand(29, 29)

    # Initialize the GCN model
    model = GCN(nfeat=7,  # Number of input features
                nclass=2,  # Number of output classes
                dropout=0.1)  # Dropout rate

    # Perform a forward pass
    output = model(input, adj)
    # Print the output shape (should be [nclass])
    print(output.size())


torch.Size([2])
